In [1]:
import os
print(os.getcwd())
os.chdir("d:/vscode_machineLearning/BEST_PROJECTS/ForbsBillionaires")
print(os.getcwd())

d:\vscode_machineLearning\BEST_PROJECTS\ForbsBillionaires\experiments\code_trail
d:\vscode_machineLearning\BEST_PROJECTS\ForbsBillionaires


## Entity

In [2]:
from pathlib import Path
from dataclasses import dataclass

In [3]:
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir:Path
    STATUS_file :str
    data_dir:Path
    schema_check:dict
    schema_dtype_check:dict

## configuration manager

In [4]:
from predictor.constatns import *
from predictor.utils import *

In [5]:
CONFIG_FILE_PATH

WindowsPath('config/config.yaml')

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):


        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        creat_dirs([self.config.artifacts_root])

    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.INDEPENDENT_FEATURES
        schema_dtype = self.schema.INDEPENDENT_FEATURES.values()

        creat_dirs([config.root_dir])
        

        data_validation_config =  DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_file=config.STATUS_FILE,
            data_dir=config.data_dir,
            schema_check=schema,
            schema_dtype_check=schema_dtype            
        )

        return data_validation_config
        
        
        

In [7]:
read_yaml(CONFIG_FILE_PATH)

[2023-09-13 12:14:27,454: INFO: utils: yaml file: config\config.yaml loaded successfully]


ConfigBox({'artifacts_root': 'artifacts', 'data_validation': {'root_dir': 'artifacts\\validation', 'data_dir': 'artifacts\\cleand_data\\billionaires.csv', 'STATUS_FILE': 'artifacts\\cleand_data\\data_validation_status.txt'}})

In [8]:
read_yaml(SCHEMA_FILE_PATH)

[2023-09-13 12:14:27,470: INFO: utils: yaml file: schema.yaml loaded successfully]


ConfigBox({'INDEPENDENT_FEATURES': {'rank': 'int64', 'full_name': 'object', 'age': 'int64', 'gender': 'object', 'country_of_citizenship': 'object', 'city_of_residence': 'object', 'business_category': 'object', 'self_made': 'bool', 'wealth_status': 'object'}, 'DEPENDENT_FEATURES': {'net_worth': 'int64'}})

In [9]:
read_yaml(PARAMS_FILE_PATH)

[2023-09-13 12:14:27,482: INFO: utils: yaml file: params.yaml loaded successfully]


ConfigBox({'key': 'val'})

## components

In [10]:
from predictor import logger
import pandas as pd

In [11]:
config = read_yaml(CONFIG_FILE_PATH)

[2023-09-13 12:14:28,253: INFO: utils: yaml file: config\config.yaml loaded successfully]


In [12]:
type(config)

box.config_box.ConfigBox

In [13]:
config.artifacts_root

'artifacts'

In [14]:
config.data_validation.data_dir

'artifacts\\cleand_data\\billionaires.csv'

In [15]:
df = pd.read_csv(config.data_validation.data_dir)

In [16]:
list(df.dtypes)

[dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('bool'),
 dtype('O')]

In [17]:
for i in df.dtypes:
    print(i)

int64
int64
int64
float64
object
float64
object
object
object
object
bool
object


In [18]:
read_yaml(SCHEMA_FILE_PATH).INDEPENDENT_FEATURES.values()

[2023-09-13 12:14:28,386: INFO: utils: yaml file: schema.yaml loaded successfully]


dict_values(['int64', 'object', 'int64', 'object', 'object', 'object', 'object', 'bool', 'object'])

In [19]:
class DataValidataion:
    def __init__(self,config_in :DataValidationConfig):
        self.config_in = config_in
    
    def validate_features(self) -> bool:
        try:
            validation_status = False
            df = pd.read_csv(self.config_in.data_dir)
            all_features = list(df.columns)
            all_dtypes = list(df.dtypes)
            all_schema = self.config_in.schema_check
            all_schema_dtypes = self.config_in.schema_dtype_check
            

            for cols in all_features:
                if cols not in all_schema:
                    validation_status = False
                    with open(self.config_in.STATUS_file,'w') as stat_file:
                        stat_file.write(f"Validation status: {validation_status}")
                
                else:
                    validation_status = True
                    with open(self.config_in.STATUS_file,'w') as f:
                        f.write(f"Validation status: {validation_status}")

            for dtypes in all_dtypes:
                if dtypes not in all_schema_dtypes:
                    validation_status = False
                    with open(self.config_in.STATUS_file,'w') as stat_file:
                        stat_file.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config_in.STATUS_file,'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        except Exception as e:
            raise CustomException(e)
        

## pipeline

In [20]:
try:
    confg = ConfigurationManager()
    data_val_confg = config.get_data_validation_config()
    data_validation = DataValidataion(config_in=data_val_confg)
    data_validation.validate_features()

except Exception as e:
    raise CustomException(e)


# try:
#     config = ConfigurationManager()
#     data_val_config = config.get_data_validation_config()
#     data_validation = DataValidation(config_in=data_val_config)
#     data_validation.validate_features()
# except Exception as e:
#     raise CustomException(e,sys)


[2023-09-13 12:14:28,414: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2023-09-13 12:14:28,415: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-09-13 12:14:28,419: INFO: utils: yaml file: schema.yaml loaded successfully]
[2023-09-13 12:14:28,420: INFO: utils: Created artifacts]


CustomException: Error occured in python script name [C:\Users\Gyanprakash\AppData\Local\Temp\ipykernel_14224\2331035770.py] line number [3] error message ["'ConfigBox' object has no attribute 'get_data_validation_config'"]

# **The `params.yaml` was Empty that's why I was getting error.**